In [1]:
import pandas as pd
import csv
import re #regular expression
from textblob import TextBlob
import string
import preprocessor as p
from preprocessor.api import clean
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
df = pd.read_csv("new-tweet-9.csv")

In [3]:
df['tweet-date']

0        11/01/17
1        11/01/17
2        11/01/17
3        11/01/17
4        11/01/17
           ...   
37316    31/12/17
37317    31/12/17
37318    31/12/17
37319    31/12/17
37320    31/12/17
Name: tweet-date, Length: 37321, dtype: object

In [ ]:
#Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

In [ ]:
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [ ]:
clean_text = []
for i in range(0,len(df['tweet'])):
    TextCleaning = clean(df['tweet'][i])
    clean_text.append(TextCleaning)

In [ ]:
clean_text = pd.DataFrame(clean_text)
clean_text.columns=["Clean-Tweet"]

In [ ]:
def clean_tweets(tweet):
 
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
#after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)

    #remove hashtags 
    tweet = re.sub("[^a-zA-Z]", " ", tweet)
    
    #remove url
    tweet = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', tweet, flags=re.MULTILINE)
    #filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)return tweet

In [ ]:
clean_text_2 = []
for i in range(0,len(clean_text['Clean-Tweet'])):
    TextCleaning = clean_tweets(df['tweet'][i])
    clean_text_2.append(TextCleaning)

In [ ]:
clean_text_2 = pd.DataFrame(clean_text_2)
clean_text_2.columns=["Clean-Tweet"]

In [ ]:
clean_text_2
clean_text_2 = clean_text_2.drop_duplicates(subset=['Clean-Tweet'], keep=False)

In [ ]:
clean_text_2=clean_text_2.reset_index()

In [ ]:
clean_text_2

In [ ]:
polarity = []
for i in range(0,len(clean_text_2['Clean-Tweet'])):
    blob = TextBlob(clean_text_2['Clean-Tweet'][i])
    Sentiment = blob.sentiment     
    print(Sentiment)
    polarity.append(Sentiment.polarity)
    subjectivity = Sentiment.subjectivity

In [ ]:
len(polarity)
clean_text_2['Twitter-Sentiment'] = pd.DataFrame(polarity)
df['BSE_Sentiment'] = clean_text_2['Twitter-Sentiment']

In [ ]:
df.to_csv("new_sentiment-9.csv")

In [ ]:
df['tweet-date'] = df['tweet-date'].astype('datetime64[D]')

In [4]:
df = df.sort_values(by='tweet-date')

In [5]:
df_new= df.groupby(['tweet-date']).mean()

In [7]:
df_new

,BSE_Sentiment
tweet-date,
11/01/17,0.070970
11/03/17,0.117208
11/04/17,0.133175
11/05/17,0.051314
11/07/17,0.106765
11/08/17,0.018866
11/09/17,-0.011064
11/10/17,0.078236
11/12/17,0.098539


In [9]:
df_new=df_new.reset_index(level=['tweet-date'])

In [11]:
df_new.to_csv("final-sentiment.csv")